In [1]:
from sklearn.model_selection import ParameterGrid
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import pandas as pd
import numpy as np
import os
from glob import glob

In [2]:
data_path = "results_sims_steps/sims"
# Hyperparameter grid
param_grid = {
    'filters': [16,32, 64],
    'kernel_size': [3,5],
    'dropout_rate': [0],
    'learning_rate': [1e-1, 1e-2, 1e-3, 1e-4],
    'batch_size': [16, 32, 64]
}

In [3]:
class CombinedArrayDataset(Dataset):
    def __init__(self, folder_path):
        # Load all CSV files in the folder
        csv_files = glob(os.path.join(folder_path, "*.csv"))
        data_frames = [pd.read_csv(file) for file in csv_files]
        combined_df = pd.concat(data_frames, ignore_index=True)

        # Parse the arrays and labels
        self.belief = combined_df['belief'].apply(lambda x: np.array(eval(x), dtype=np.float32)).tolist()
        self.ship = combined_df['ship'].apply(lambda x: np.array(eval(x), dtype=np.float32)).tolist()
        self.labels = combined_df['remain'].tolist()

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        # Get the array and label for the given index
        belief = self.belief[idx]
        ship = self.ship[idx]
        label = self.labels[idx]

        # Convert to PyTorch tensors
        belief = torch.tensor(belief, dtype=torch.float32)  # Ensure float32 for features
        ship = torch.tensor(ship, dtype= torch.float32)
        label = torch.tensor(label, dtype=torch.float32)  # Ensure long for labels

        # Combine belief and ship as two channels
        input_tensor = torch.stack((belief, ship), dim=0)  # Shape: (2, height, width)
        return input_tensor, label

# Example usage

dataset = CombinedArrayDataset(data_path)


KeyboardInterrupt: 

In [ ]:
# Assuming you have a PyTorch model class defined as `MyCNNModel`
class MyCNNModel(nn.Module):
    def __init__(self, filters, kernel_size, strides, dropout_rate):
        super(MyCNNModel, self).__init__()
        self.conv1 = nn.Conv2d(2, filters, kernel_size, stride=strides, padding = 1)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        
        # Calculate fc1 input size
         # Calculate the size after 2 Conv layers and pooling
        # After conv1: (30 - 3 + 2 * 1) / 1 + 1 = 30
        # After pool: 30 / 2 = 15
        # After conv2: (15 - 3 + 2 * 1) / 1 + 1 = 15
        # After pool: 15 / 2 = 7
        # conv1_output = (30-filters+2*1)//strides + 1
        # after_pool1 = conv1_output//2 + 1
        # conv2_output = (after_pool2-filters+2*1)//strides + 1
        # after_pool2 = conv2_output//2 + 1
        # self.fc_input_size = filters * 2 * 7 * 7  # Updated input size after conv2 and pool
        
        conv_out_size = ((30 - kernel_size + 2) // strides) + 1  
        pool_out_size = ((conv_out_size - 2) // 2) + 1  
        fc1_input_size = filters * (pool_out_size ** 2)

        self.fc1 = nn.Linear(fc1_input_size, 256)  # Adjust based on input size
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128,1)
        self.dropout = nn.Dropout(dropout_rate)
        self.flatten = nn.Flatten()
    
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.flatten(x)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x


In [ ]:
# Device configuration
device = torch.device("mps" if torch.has_mps else "cpu")
print(f"using device: {device}")
# Placeholder for results
results = []

# Dataset (assuming `train_dataset` and `test_dataset` are already defined)

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Create DataLoaders


for params in ParameterGrid(param_grid):
    print(f"Training with parameters: {params}")
    train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=params['batch_size'], shuffle=False)

    # Initialize model
    model = MyCNNModel(params['filters'], params['kernel_size'],params['strides'] ,params['dropout_rate']).to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

    # Train model
    num_epochs = 25
    train_losses = []
    test_losses = []
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs.squeeze(), labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        test_loss = 0.0
        with torch.no_grad():  # No need to compute gradients during testing
            for inputs, targets in test_loader:
                inputs, targets = inputs.to(device), targets.to(device)
                # Forward pass
                outputs = model(inputs)
                
                # Compute the loss
                loss = criterion(outputs.squeeze(), targets)
                test_loss += loss.item()

        avg_test_loss = test_loss / len(test_loader)
        test_losses.append(avg_test_loss)
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}, Test Loss: {test_loss / len(test_loader):.4f}")

    # Evaluate model on validation data
    # model.eval()
    # correct, total = 0, 0
    # with torch.no_grad():
    #     for images, labels in val_loader:
    #         images, labels = images.to(device), labels.to(device)
    #         outputs = model(images)
    #         _, predicted = torch.max(outputs, 1)
    #         total += labels.size(0)
    #         correct += (predicted == labels).sum().item()
    # val_accuracy = correct / total

    # Test model
    test_loss, total = 0, 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            # _, predicted = torch.max(outputs, 1)
            # total += labels.size(0)
            # correct += (predicted == labels).sum().item()
    # test_accuracy = correct / total
            loss = criterion(outputs.squeeze(), labels)
            test_loss += loss.item()

        avg_test_loss = test_loss / len(test_loader)
        # test_losses.append(avg_test_loss)
    # Save results
    results.append((params, avg_train_loss,  avg_test_loss))
    print(f"Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}")

/var/folders/j2/56dmz8kj7h5fyr879140wlzr0000gn/T/ipykernel_72000/1481307314.py:2: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  device = torch.device("mps" if torch.has_mps else "cpu")


using device: mps
Training with parameters: {'batch_size': 16, 'dropout_rate': 0, 'filters': 32, 'kernel_size': 3, 'learning_rate': 0.1, 'strides': 1}
Epoch [1/50], Loss: 16194.9471, Test Loss: 3248.9178
Epoch [2/50], Loss: 3292.5617, Test Loss: 3300.7354
Epoch [3/50], Loss: 3288.8790, Test Loss: 3251.8031
Epoch [4/50], Loss: 3286.0268, Test Loss: 3247.1091
Epoch [5/50], Loss: 3285.3208, Test Loss: 3251.6914
Epoch [6/50], Loss: 3284.8167, Test Loss: 3251.4483
Epoch [7/50], Loss: 3285.5276, Test Loss: 3251.2041
Epoch [8/50], Loss: 3283.7909, Test Loss: 3247.1965
Epoch [9/50], Loss: 3283.5549, Test Loss: 3268.7253
Epoch [10/50], Loss: 3285.0224, Test Loss: 3247.1959
Epoch [11/50], Loss: 3282.4108, Test Loss: 3270.2702
Epoch [12/50], Loss: 3281.5287, Test Loss: 3271.5888
Epoch [13/50], Loss: 3283.2140, Test Loss: 3247.9420
Epoch [14/50], Loss: 3281.4881, Test Loss: 3247.5062
Epoch [15/50], Loss: 3280.3637, Test Loss: 3248.3813
Epoch [16/50], Loss: 3278.6056, Test Loss: 3247.3535
Epoch [17

KeyboardInterrupt: 

In [ ]:
results

[({'batch_size': 16,
   'dropout_rate': 0,
   'filters': 32,
   'kernel_size': 3,
   'learning_rate': 0.1,
   'strides': 1},
  [16194.947056189074,
   3292.561664282486,
   3288.8790442503387,
   3286.0267736976107,
   3285.320848820859,
   3284.8166698140776,
   3285.527627116817,
   3283.790940905592,
   3283.5548529387106,
   3285.022400261668,
   3282.410796245784,
   3281.528735421431,
   3283.213966939806,
   3281.4881408841125,
   3280.3637024175314,
   3278.605589203355,
   3275.4491643723172,
   3275.0328355865604,
   3275.1365915892657,
   3275.8974027270287,
   3275.3032936227196,
   3275.1655106152266,
   3275.1831530392415,
   3275.435699637562,
   3275.393238662133,
   3275.246429009176,
   3275.1776739686607,
   3275.4425212520123,
   3275.4410285906038,
   3275.138917522789,
   3275.169674101211,
   3275.4949075918753,
   3275.170228128753,
   3275.4271265786297,
   3275.3113789294066,
   3275.2178020436745,
   3275.3286345512283,
   3275.5042083415847,
   3275.61845750

In [ ]:
# Save results to a file
results_file = "pytorch_grid_search_results.txt"

with open(results_file, 'w') as file:
    file.write("Parameters | Train Loss| Test Loss\n")
    file.write("-" * 50 + "\n")
    for params, train_loss, test_loss in results:
        param = ""
        for item,val in params.items():
            param += f"{str(item)}:{str(val)}, "
        file.write(f"{param} | {train_loss[-1]:.4f} | {test_loss:.4f}\n")

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read results from the file
results_df = pd.read_csv(results_file, sep="|", skipinitialspace=True)

# Plot results
plt.figure(figsize=(10, 6))
plt.plot(results_df.index, results_df['Validation Accuracy'], label='Validation Accuracy', marker='o')
plt.plot(results_df.index, results_df['Test Accuracy'], label='Test Accuracy', marker='x')
plt.xlabel('Parameter Combinations')
plt.ylabel('Accuracy')
plt.title('Comparison of Validation and Test Accuracy')
plt.legend()
plt.grid(True)
plt.xticks(results_df.index, results_df['Parameters'], rotation=45, ha='right', fontsize=8)
plt.tight_layout()
plt.show()